<a href="https://colab.research.google.com/github/pakdaniel/MECS-4995-AML-Group-24/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


pth = "met_gas_site_cleaned.snappy.parquet"
def generate_datasets(pth):

  df = pd.read_parquet(pth)
  df["year"] = df["DATE_TIME"].dt.year
  df["month"] = df["DATE_TIME"].dt.month
  df["LATITUDE"] = df["LATITUDE"].astype("float")
  df["LONGITUDE"] = df["LONGITUDE"].astype("float")

  categorical_columns = ['LAND_USE', 'TERRAIN']
  for feature in categorical_columns:
      encoder = OneHotEncoder()
      encoded_data = encoder.fit_transform(df[[feature]])
      df[encoder.categories_[0]] = encoded_data.toarray()


  drop_columns = ['OZONE', "year", "DATE_TIME", "SITE_ID", "QA_CODE", "WINDSPEED", "SIGMA_THETA"] + categorical_columns
  df_train = df[df['year'].isin([2013, 2014, 2015, 2016, 2017, 2018])]
  X_train, y_train = df_train.drop(columns=drop_columns, axis=1), df_train['OZONE']

  columns = X_train.columns

  numerical_features = ['TEMPERATURE',
                        'RELATIVE_HUMIDITY',
                        'SOLAR_RADIATION',
                        'PRECIPITATION',
                        'WIND_DIRECTION',
                        'FLOW_RATE',
                        'WINDSPEED_SCALAR',
                        'SHELTER_TEMPERATURE',
                        'NO',
                        'NOY',
                        'NOYDIF',
                        'SO2_GA',
                        'LATITUDE',
                        'LONGITUDE',
                        'ELEVATION']

  ct = ColumnTransformer([
          ('Standardize Numerical Features', StandardScaler(), numerical_features)
      ], remainder='passthrough')
  X_train = ct.fit_transform(X_train)

  df_val = df[df['year'].isin([2019, 2020])]
  X_val, y_val = df_val.drop(columns=drop_columns, axis=1), df_val['OZONE']
  X_val = ct.transform(X_val)

  df_test = df[df['year'].isin([2021, 2022])]
  X_test, y_test = df_test.drop(columns=drop_columns, axis=1), df_test['OZONE']
  X_test = ct.transform(X_test)

  return columns, X_train, y_train, X_val, y_val, X_test, y_test

columns, X_train, y_train, X_val, y_val, X_test, y_test = generate_datasets("met_gas_site_cleaned.snappy.parquet")

In [65]:
columns

Index(['TEMPERATURE', 'RELATIVE_HUMIDITY', 'SOLAR_RADIATION', 'PRECIPITATION',
       'WIND_DIRECTION', 'FLOW_RATE', 'WINDSPEED_SCALAR',
       'SHELTER_TEMPERATURE', 'NO', 'NOY', 'NOYDIF', 'SO2_GA', 'LATITUDE',
       'LONGITUDE', 'ELEVATION', 'month', 'Agric', 'Forest', 'Range',
       'Complex', 'Flat', 'Rolling'],
      dtype='object')